In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '0'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from datasets import load_dataset

dataset = load_dataset("LeoTungAnh/kdd210_hourly")
dataset

DatasetDict({
    train: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    test: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
})

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
len(test_dataset[0]['target'])

10898

In [8]:
type(test_dataset)

datasets.arrow_dataset.Dataset

In [9]:
import pandas as pd

# Create a list of dictionaries, where each dictionary represents a row in the DataFrame
df = pd.DataFrame()

for row in test_dataset:
  df[row['item_id']] = row['target']

df['date'] = '2015-01-01 00:00:00'
df = df.set_index("date")
df.head()

/tmp/ipykernel_4085075/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085075/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085075/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T201,T202,T203,T204,T205,T206,T207,T208,T209,T210
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,1.468122,1.538769,-0.030617,-0.781501,-0.802695,-0.772417,-0.378809,-0.237514,-0.393948,-0.810264,...,-0.131542,-0.785538,-0.807741,-0.777464,1.493353,1.654833,-0.025571,-0.776959,-0.777464,-0.792602
2015-01-01 00:00:00,1.316855,1.449373,-0.079674,-0.773862,-0.798327,-0.767746,-0.512904,0.000000,-0.507808,-0.803933,...,-0.140836,-0.775391,-0.798327,-0.772842,1.444276,1.724601,-0.023609,-0.804443,-0.767746,-0.783036
2015-01-01 00:00:00,1.261700,1.636722,-0.061296,-0.759776,-0.780089,-0.754046,-0.639456,-0.582161,-0.597787,-0.792069,...,-0.134217,-0.759776,-0.780089,-0.754046,1.501297,1.772147,-0.009210,-0.791548,-0.748838,-0.769672
2015-01-01 00:00:00,1.487590,1.831615,-0.022891,-0.730291,-0.753942,-0.721690,-0.646435,-0.560429,-0.555053,-0.765768,...,-0.114272,-0.733516,-0.753942,-0.732441,0.525398,0.000000,-0.270158,-0.744267,-0.678687,-0.748567
2015-01-01 00:00:00,1.842409,2.225288,0.150884,-0.702879,-0.723452,-0.694879,-0.632018,-0.500582,-0.454865,-0.741738,...,-0.066271,-0.707451,-0.729166,-0.717737,-0.672020,0.000000,-0.569157,-0.743453,-0.557728,-0.723452


In [10]:
# # Save the DataFrame to a CSV file
# df.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/kdd/kdd.csv', index_label='date')

In [11]:
df.shape

(10898, 210)

In [12]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(df.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/kdd/'
args.data_path = 'kdd.csv'
args.model_id = 'kdd_96_96'
args.data_path

'kdd.csv'

In [13]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [14]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [15]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(21, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(21, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
          

In [16]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on KDD dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on KDD dataset",
    # Hyperparams
    config={
        "dataset": "KDD210",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 210,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 125.74700236320496
Epoch: 2 cost time: 143.6630027294159
Epoch: 3 cost time: 151.66661739349365
Epoch: 4 cost time: 154.90364527702332
Epoch: 5 cost time: 163.54127597808838


Progress:   3%|▎         | 1/30 [13:56<6:44:06, 836.09s/it]

Epoch: 1 cost time: 164.94005465507507
Epoch: 2 cost time: 166.4779760837555
Epoch: 3 cost time: 164.68456959724426
Epoch: 4 cost time: 164.82093024253845
Epoch: 5 cost time: 163.94800877571106


Progress:   7%|▋         | 2/30 [29:16<6:53:20, 885.72s/it]

...

Progress:  93%|█████████▎| 28/30 [7:06:27<30:31, 915.73s/it]

Epoch: 1 cost time: 163.78876543045044
Epoch: 2 cost time: 163.4000072479248
Epoch: 3 cost time: 163.13321828842163
Epoch: 4 cost time: 164.3012924194336
Epoch: 5 cost time: 163.48534440994263


Progress:  97%|█████████▋| 29/30 [7:21:41<15:15, 915.22s/it]

Epoch: 1 cost time: 164.70007252693176
Epoch: 2 cost time: 163.7963650226593
Epoch: 3 cost time: 163.2300682067871
Epoch: 4 cost time: 164.23265743255615
Epoch: 5 cost time: 164.39813661575317


Progress: 100%|██████████| 30/30 [7:36:56<00:00, 913.89s/it]
